# analyzing dense output
Timothy Tyree<br>
3.26.2021

In [1]:
# darkmode=True
from lib.my_initialization import *
# For darkmode plots
from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)


## plot the collision times

In [155]:
#load the example data
# os.chdir(nb_dir)
# data_dir='data/out.csv'
# assert ( os.path.exists(data_dir) ) 
# df=pd.read_csv(data_dir,lineterminator=None)

#load a log file
input_folder=f"{nb_dir}/data"
input_fn='example.log'
os.chdir(input_folder)
df=parse_output_log(input_fn, include_inputs=False,printing=False)
Ntrials=df.index.values.shape[0]
Nmin=11
df.drop(columns=[f'{i}' for i in range(Nmin)], inplace=True)
#any duds still there?
assert ( not (df.values<-9000).any() )
df.head()

11       12       13       14       15       16       17       18  \
0  2.47000  2.47000  2.47000  2.47000  2.47000  2.47000  0.69080  0.56114   
1  1.18204  1.18204  1.18204  1.18204  1.18204  1.18204  1.18204  1.18204   
2  0.81593  0.81593  0.81593  0.81593  0.81593  0.81593  0.81593  0.81593   
3  0.14193  0.14193  0.14193  0.14193  0.14193  0.14193  0.14193  0.14193   
4  2.43258  2.43258  2.43258  2.43258  2.43258  2.43258  2.43258  2.43258   

        19       20  ...       60       61       62       63       64  \
0  0.56114  0.56114  ...  0.06454  0.06454  0.06454  0.06454  0.06454   
1  1.18204  1.18204  ...  0.10980  0.10980  0.02032  0.02032  0.02032   
2  0.81593  0.81593  ...  0.03425  0.03425  0.03425  0.03425  0.03425   
3  0.14193  0.14193  ...  0.03570  0.03570  0.03570  0.03570  0.02346   
4  2.43258  2.43258  ...  0.00877  0.00877  0.00877  0.00877  0.00877   

        65       66       67       68       69  
0  0.06454  0.06454  0.06454  0.06454  0.02795  
1  0.02032  0.02032  0.02032  0.02032  0.02032  
2  0.03425  0.03425  0.03425  0.03425  0.03425  
3  0.02346  0.02346  0.02346  0.02346  0.02346  
4  0.00877  0.00877  0.00877  0.00877  0.00877  

[5 rows x 59 columns]

In [156]:
dfd=df.describe().T.drop(columns=['count'])
dfd.head()

mean       std      min       25%       50%       75%      max
11  1.282126  1.340333  0.00269  0.311847  0.852540  1.832960  7.00783
12  1.090162  1.123887  0.00269  0.276240  0.735680  1.507798  6.12881
13  0.898572  0.930067  0.00269  0.261888  0.639650  1.217553  5.53320
14  0.776358  0.756320  0.00269  0.222100  0.601945  1.057830  4.84248
15  0.660640  0.661866  0.00108  0.172733  0.502590  0.893335  3.97202

In [157]:
#extract the values
x_values=np.array([eval(x) for x in dfd.index.values])
y_values=np.array(dfd['50%'].values)
yerr1_values=np.array(dfd['25%'].values)
yerr2_values=np.array(dfd['75%'].values)

y2_values=np.array(dfd['mean'].values)
y2err1_values=y2_values-np.array(dfd['std'].values)
y2err2_values=y2_values+np.array(dfd['std'].values)

In [158]:
#plot the values
figsize=(12,6)
fontsize=18
saving=True
savefig_folder=f'{nb_dir}/../fig'
savefig_fn='colltimes.png'
fig, axs = plt.subplots(ncols=2,figsize=figsize)
ax=axs[1]
ax.fill_between(x_values,yerr1_values,yerr2_values,alpha=0.3)
ax.plot(x_values,y_values,lw=2)
ax.set_xlabel('n', fontsize=fontsize)
ax.set_ylabel(r'time to first collision (sec)', fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)
ax.tick_params(axis='both', which='minor', labelsize=0)
ax.set_title('median', fontsize=fontsize)
ax=axs[0]
ax.fill_between(x_values,y2err1_values,y2err2_values,alpha=0.3)
ax.plot(x_values,y2_values,lw=2)
ax.set_xlabel('n', fontsize=fontsize)
ax.set_ylabel(r'time to first collision (sec)', fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)
ax.tick_params(axis='both', which='minor', labelsize=0)
ax.set_title('mean', fontsize=fontsize)
fig.tight_layout()
if not saving:
    plt.show()
else:
    plt.tight_layout()
    os.chdir(savefig_folder)
    plt.savefig(savefig_fn, dpi=300)
    print(f"saved figure in \n\t{savefig_fn}")
#     plt.close()

saved figure in 
	colltimes.png


## plot the collision rates

In [163]:
#plot the values
figsize=(12,6)
fontsize=18
saving=True
savefig_folder=f'{nb_dir}/../fig'
savefig_fn='collrates.png'
fig, axs = plt.subplots(ncols=2,figsize=figsize)
ax=axs[1]
ax.fill_between(x_values,1/yerr2_values,1/yerr1_values,alpha=0.3)
ax.plot(x_values,1/y_values,lw=2)
ax.set_xlabel('n', fontsize=fontsize)
ax.set_ylabel(r'$W_{{-2}}$ (1/sec)', fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)
ax.tick_params(axis='both', which='minor', labelsize=0)
ax.set_title('median', fontsize=fontsize)
ax=axs[0]
# ax.fill_between(x_values,1/y2err2_values,1/y2err1_values,alpha=0.3)
ax.plot(x_values,1/y2_values,lw=2)
ax.set_xlabel('n', fontsize=fontsize)
ax.set_ylabel(r'$W_{{-2}}$ (1/sec)', fontsize=fontsize)
ax.tick_params(axis='both', which='major', labelsize=fontsize)
ax.tick_params(axis='both', which='minor', labelsize=0)
# ax.set_ylim([0,1])
ax.set_title('mean', fontsize=fontsize)
fig.tight_layout()
if not saving:
    plt.show()
else:
    plt.tight_layout()
    os.chdir(savefig_folder)
    plt.savefig(savefig_fn, dpi=300)
    print(f"saved figure in \n\t{savefig_fn}")
#     plt.close()

saved figure in 
	collrates.png


## Results
collision rates/times are being generated with uncertainty that cannot be reliably quantified in terms of mean and standard deviation. Therefore, output logs must include dense output

- DONE: dev parsing output logs
- DONE: generate run_1.dat and run_test.dat


In [144]:
def parse_output_log(input_fn,include_inputs=True,printing=False):
    with open(input_fn) as f:
        trgt1='Printing Inputs...\n'
        trgt2='Printing Outputs...\n'
        for n,line in enumerate(f):
            if trgt1 == line:
                if printing:
                    print(f'found inputs starting after line {n}')
                n_input=n
            if trgt2 == line:
                if printing:
                    print(f'found outputs starting after line {n}')
                n_output=n

    with open(input_fn) as f:
        inputs=f.readlines()[n_input+1:n_output-1]
    col_name_lst=[]
    col_value_lst=[]
    for line in inputs:
        string=line.split(' ')[-1]
        eid=string.find('=')
        if eid!=-1:
            col_name=string[:eid]
            col_value=eval(string[eid+1:-2])
            col_name_lst.append(col_name)
            col_value_lst.append(col_value)
    df=pd.read_csv(input_fn,header=n_output-2)
    #drop that 'Unammed: {Nmax}' column 
    df.drop(columns=[df.columns[-1]], inplace=True)
    if include_inputs:
        if printing:
            print("input parameters were:")
            print(col_name_lst)
            print(col_value_lst)
            print("returning outputs as pandas.DataFrame instance")
        for name,value in zip ( col_name_lst, col_value_lst):
            df[name]=value
    return df

In [147]:
input_folder=f"{nb_dir}/data"
input_fn='example.log'
os.chdir(input_folder)
df=parse_output_log(input_fn, include_inputs=True,printing=True)
df.head()

found inputs starting after line 10
found outputs starting after line 17
input parameters were:
['r', 'D', 'L', 'kappa', 'dt']
[5.0, 1.56, 2.025, 1.0, 1.0]
returning outputs as pandas.DataFrame instance


0           1           2           3           4           5  \
0  -9.99900e+03 -9999.00000 -9999.00000 -9999.00000 -9999.00000 -9999.00000   
1  -9.99900e+03 -9999.00000 -9999.00000 -9999.00000 -9999.00000 -9999.00000   
2  -9.99900e+03 -9999.00000 -9999.00000 -9999.00000 -9999.00000 -9999.00000   
3  -9.99900e+03     0.14193     0.14193     0.14193     0.14193     0.14193   
4  -9.99900e+03 -9999.00000 -9999.00000 -9999.00000 -9999.00000 -9999.00000   

            6           7           8           9  ...       65       66  \
0 -9999.00000 -9999.00000 -9999.00000     2.47000  ...  0.06454  0.06454   
1 -9999.00000 -9999.00000 -9999.00000     1.18204  ...  0.02032  0.02032   
2 -9999.00000 -9999.00000 -9999.00000 -9999.00000  ...  0.03425  0.03425   
3     0.14193     0.14193     0.14193     0.14193  ...  0.02346  0.02346   
4 -9999.00000 -9999.00000 -9999.00000 -9999.00000  ...  0.00877  0.00877   

        67       68       69    r     D      L  kappa   dt  
0  0.06454  0.06454  0.02795  5.0  1.56  2.025    1.0  1.0  
1  0.02032  0.02032  0.02032  5.0  1.56  2.025    1.0  1.0  
2  0.03425  0.03425  0.03425  5.0  1.56  2.025    1.0  1.0  
3  0.02346  0.02346  0.02346  5.0  1.56  2.025    1.0  1.0  
4  0.00877  0.00877  0.00877  5.0  1.56  2.025    1.0  1.0  

[5 rows x 75 columns]

## generate run_1.dat 


In [167]:
A_values=np.array([20.25,25,39,50,56.25,100,156.25,189])[::-1]
L_values=np.sqrt(A_values)
L_values

array([13.74772708, 12.5       , 10.        ,  7.5       ,  7.07106781,
        6.244998  ,  5.        ,  4.5       ])

In [183]:
niter=250 #trials per worker
r_values=np.array([0.1,0.2,0.3,0.4,0.5,1.0])#cm
D_values=np.array([0.25,0.5,0.75,1.0,1.25,1.5,1.75,2.0])#cm^2/s
A_values=np.array([20.25,25,39,50,56.25,100,156.25,189])[::-1]#cm^2
L_values=np.sqrt(A_values)#cm
kappa_values=np.array([1,10,100])#1/s
num_trials_per_setting=6
#iterate over settings, scheduling the longest jobs first
count=0
for r in r_values:
    for D in D_values:
        for L in L_values:
            for kappa in kappa_values:
                num_trials=0
                while num_trials<num_trials_per_setting:
                    num_trials+=1
                    count=count+1
print(count)

6912


## debugging log parsing

In [16]:
input_fn='/home/timothytyree/Documents/GitHub/bgmc/python/data/osg_output/Log/job.out.8623404.5130'
# def parse_output_log(input_fn,
include_inputs=True,
printing=True
# ):
with open(input_fn) as f:
    trgt1='Printing Inputs...\n'
    trgt2='Printing Outputs...\n'
    for n,line in enumerate(f):
        if trgt1 == line:
            if printing:
                print(f'found inputs starting after line {n}')
            n_input=n
        if trgt2 == line:
            if printing:
                print(f'found outputs starting after line {n}')
            n_output=n

with open(input_fn) as f:
    inputs=f.readlines()[n_input+1:n_output-1]
col_name_lst=[]
col_value_lst=[]
for line in inputs:
    string=line.split(' ')[-1]
    eid=string.find('=')
    if eid!=-1:
        col_name=string[:eid]
        col_value=eval(string[eid+1:-1])
        col_name_lst.append(col_name)
        col_value_lst.append(col_value)
df=pd.read_csv(input_fn,header=n_output-1)
#drop that 'Unammed: {Nmax}' column 
df.drop(columns=[df.columns[-1]], inplace=True)
if include_inputs:
    if printing:
        print("input parameters were:")
        print(col_name_lst)
        print(col_value_lst)
        print("returning outputs as pandas.DataFrame instance")
    for name,value in zip ( col_name_lst, col_value_lst):
        df[name]=value
#     return df

found inputs starting after line 11
found outputs starting after line 18


NameError: name 'pd' is not defined

In [9]:
eid=string.find('=')
if eid!=-1:
    col_name=string[:eid]
    col_value=eval(string[eid+1:-2])
    col_name_lst.append(col_name)
    col_value_lst.append(col_value)

SyntaxError: unexpected EOF while parsing (<string>, line 0)

In [15]:
string[eid+1:-1]

'1'

# (not meant for this ipynb) analyze manual output

In [2]:
#preliminary?
arr1=np.array([6.62549e-05,6.329412e-05,6.154902e-05,5.390196e-05,4.927451e-05,4.609804e-05,4.245098e-05,4.035294e-05,3.492157e-05,2.927451e-05,2.776471e-05,2.539216e-05,2.288235e-05,1.94902e-05,1.717647e-05,1.590196e-05,1.413725e-05,1.362745e-05,1.109804e-05,1.011765e-05,9.235294e-06,8.941176e-06,7.941176e-06,7.039216e-06,6.313725e-06,6.176471e-06,6.078431e-06,6e-06,5.823529e-06,5.45098e-06,4.529412e-06,4.509804e-06,4.352941e-06,4.313725e-06,4e-06,3.921569e-06,3.862745e-06,3.745098e-06,3.588235e-06,3.411765e-06,3.333333e-06,2.627451e-06,2.529412e-06,2.490196e-06,2.313725e-06,2.313725e-06,2.294118e-06,2.27451e-06,2.254902e-06,2.078431e-06,1.960784e-06,1.803922e-06,1.72549e-06,1.705882e-06,1.647059e-06,1.627451e-06,1.588235e-06,1.568627e-06,1.568627e-06,1.529412e-06,1.45098e-06,1.45098e-06,1.45098e-06,1.45098e-06,1.45098e-06,1.431373e-06,1.431373e-06,1.431373e-06,1.431373e-06,1.431373e-06,1.411765e-06,1.392157e-06,1.392157e-06,1.352941e-06,1.333333e-06,1.333333e-06,1.313725e-06,1.313725e-06,1.27451e-06,1.215686e-06,1.215686e-06,1.176471e-06,1.176471e-06,1.176471e-06,1.176471e-06,1.176471e-06,1.176471e-06,1.176471e-06,1.176471e-06,1.137255e-06,1.137255e-06,1.137255e-06,1.137255e-06,1.137255e-06,1.137255e-06,1.117647e-06,1.098039e-06,1.078431e-06,1.078431e-06,1.078431e-06,1.078431e-06,1.078431e-06,1.078431e-06,1.078431e-06,1.078431e-06,1.078431e-06,1.078431e-06,1.078431e-06,1.078431e-06,1.078431e-06,1.058824e-06,1.058824e-06])
arr2=np.array([4.143137e-05,4.027451e-05,4.027451e-05,4.015686e-05,3.792157e-05,3.439216e-05,2.94902e-05,2.560784e-05,2.513725e-05,2.264706e-05,2.058824e-05,1.770588e-05,1.670588e-05,1.55098e-05,1.523529e-05,1.347059e-05,1.168627e-05,1.009804e-05,9.529412e-06,9.058824e-06,8.431373e-06,8e-06,7.666667e-06,7.666667e-06,7.215686e-06,6.509804e-06,5.901961e-06,5.666667e-06,5e-06,4.627451e-06,4.156863e-06,3.960784e-06,3.784314e-06,3.72549e-06,3.607843e-06,3.529412e-06,3.529412e-06,3.333333e-06,3.313725e-06,3.294118e-06,3.039216e-06,2.980392e-06,2.745098e-06,2.490196e-06,2.352941e-06,2.333333e-06,2.333333e-06,2.333333e-06,2.235294e-06,2.215686e-06,2.137255e-06,2.039216e-06,1.960784e-06,1.921569e-06,1.882353e-06,1.882353e-06,1.862745e-06,1.803922e-06,1.72549e-06,1.705882e-06,1.666667e-06,1.490196e-06,1.333333e-06,1.333333e-06,1.313725e-06,1.294118e-06,1.27451e-06,1.27451e-06,1.254902e-06,1.254902e-06,1.254902e-06,1.254902e-06,1.254902e-06,1.254902e-06,1.235294e-06,1.235294e-06,1.235294e-06,1.215686e-06,1.215686e-06,1.215686e-06,1.196078e-06,1.176471e-06,1.156863e-06,1.156863e-06,1.117647e-06,1.117647e-06,1.117647e-06,1.117647e-06,1.117647e-06,1.117647e-06,1.117647e-06,1.098039e-06,1.098039e-06,1.058824e-06,1.039216e-06,1.039216e-06,1.039216e-06,1.039216e-06,1.039216e-06,1.039216e-06,1.039216e-06,1.039216e-06,1.039216e-06,1.039216e-06,1.039216e-06,1.039216e-06,1.039216e-06,1.039216e-06,1.039216e-06,1.019608e-06,1.019608e-06,1.019608e-06,1.019608e-06,1.019608e-06,1.019608e-06,1.019608e-06])

In [5]:
# test-3-7.input all equal :( niter=30
arr1=np.array([0.00263,0.001522,0.002293333,0.002127857,0.00177,0.001630476,0.001752917,0.001701538,0.001588077,0.001218889,0.001012857,0.0008446429,0.0007289655,0.0006366667,0.0005946667,0.000502,0.0004183333,0.000389,0.000378,0.0003736667,0.000344,0.00033,0.0003193333,0.0003073333,0.000291,0.0002713333,0.000257,0.0002026667,0.000162,0.0001593333,0.0001453333,0.0001386667,0.0001386667,0.0001386667,0.00012,0.0001093333,0.000109,0.0001063333,0.0001036667,9.266667e-05,8.633333e-05,8.633333e-05,8.566667e-05,8.1e-05,5.766667e-05,4.733333e-05,3.833333e-05,3.833333e-05,3.833333e-05,3.833333e-05,3.533333e-05,3.466667e-05,3.466667e-05,3.466667e-05,3.466667e-05,3.466667e-05,3.266667e-05,3.133333e-05,3.133333e-05,3.133333e-05,3.133333e-05,3.133333e-05,3.133333e-05,3.133333e-05,2.933333e-05,2.933333e-05,2.933333e-05,2.933333e-05,2.933333e-05,2.766667e-05,2.766667e-05,2.7e-05,2.633333e-05,2.633333e-05,2.566667e-05,2.533333e-05,2.5e-05,2.5e-05,2.5e-05,2.5e-05,2.5e-05,2.4e-05,2.4e-05,2.366667e-05,2.333333e-05,2.333333e-05,2.166667e-05,2.1e-05,2.1e-05,2.1e-05,2.1e-05,2.033333e-05,2e-05,1.966667e-05,1.966667e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.733333e-05,1.733333e-05,1.666667e-05,1.666667e-05,1.633333e-05,1.633333e-05,1.6e-05,1.6e-05,1.6e-05,1.566667e-05,1.566667e-05,1.566667e-05,1.566667e-05,1.566667e-05,1.533333e-05,1.466667e-05,1.466667e-05,1.466667e-05,1.466667e-05,1.466667e-05,1.4e-05,1.3e-05,1.3e-05,1.3e-05,1.3e-05,1.3e-05,1.3e-05,1.3e-05,1.3e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.233333e-05,1.2e-05,1.133333e-05,1.133333e-05,1.133333e-05,1.133333e-05,1.1e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05])
arr2=np.array([0.00263,0.001522,0.002293333,0.002127857,0.00177,0.001630476,0.001752917,0.001701538,0.001588077,0.001218889,0.001012857,0.0008446429,0.0007289655,0.0006366667,0.0005946667,0.000502,0.0004183333,0.000389,0.000378,0.0003736667,0.000344,0.00033,0.0003193333,0.0003073333,0.000291,0.0002713333,0.000257,0.0002026667,0.000162,0.0001593333,0.0001453333,0.0001386667,0.0001386667,0.0001386667,0.00012,0.0001093333,0.000109,0.0001063333,0.0001036667,9.266667e-05,8.633333e-05,8.633333e-05,8.566667e-05,8.1e-05,5.766667e-05,4.733333e-05,3.833333e-05,3.833333e-05,3.833333e-05,3.833333e-05,3.533333e-05,3.466667e-05,3.466667e-05,3.466667e-05,3.466667e-05,3.466667e-05,3.266667e-05,3.133333e-05,3.133333e-05,3.133333e-05,3.133333e-05,3.133333e-05,3.133333e-05,3.133333e-05,2.933333e-05,2.933333e-05,2.933333e-05,2.933333e-05,2.933333e-05,2.766667e-05,2.766667e-05,2.7e-05,2.633333e-05,2.633333e-05,2.566667e-05,2.533333e-05,2.5e-05,2.5e-05,2.5e-05,2.5e-05,2.5e-05,2.4e-05,2.4e-05,2.366667e-05,2.333333e-05,2.333333e-05,2.166667e-05,2.1e-05,2.1e-05,2.1e-05,2.1e-05,2.033333e-05,2e-05,1.966667e-05,1.966667e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.733333e-05,1.733333e-05,1.666667e-05,1.666667e-05,1.633333e-05,1.633333e-05,1.6e-05,1.6e-05,1.6e-05,1.566667e-05,1.566667e-05,1.566667e-05,1.566667e-05,1.566667e-05,1.533333e-05,1.466667e-05,1.466667e-05,1.466667e-05,1.466667e-05,1.466667e-05,1.4e-05,1.3e-05,1.3e-05,1.3e-05,1.3e-05,1.3e-05,1.3e-05,1.3e-05,1.3e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.233333e-05,1.2e-05,1.133333e-05,1.133333e-05,1.133333e-05,1.133333e-05,1.1e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05])
arr1=np.array([0.00263,0.001522,0.002293333,0.002127857,0.00177,0.001630476,0.001752917,0.001701538,0.001588077,0.001218889,0.001012857,0.0008446429,0.0007289655,0.0006366667,0.0005946667,0.000502,0.0004183333,0.000389,0.000378,0.0003736667,0.000344,0.00033,0.0003193333,0.0003073333,0.000291,0.0002713333,0.000257,0.0002026667,0.000162,0.0001593333,0.0001453333,0.0001386667,0.0001386667,0.0001386667,0.00012,0.0001093333,0.000109,0.0001063333,0.0001036667,9.266667e-05,8.633333e-05,8.633333e-05,8.566667e-05,8.1e-05,5.766667e-05,4.733333e-05,3.833333e-05,3.833333e-05,3.833333e-05,3.833333e-05,3.533333e-05,3.466667e-05,3.466667e-05,3.466667e-05,3.466667e-05,3.466667e-05,3.266667e-05,3.133333e-05,3.133333e-05,3.133333e-05,3.133333e-05,3.133333e-05,3.133333e-05,3.133333e-05,2.933333e-05,2.933333e-05,2.933333e-05,2.933333e-05,2.933333e-05,2.766667e-05,2.766667e-05,2.7e-05,2.633333e-05,2.633333e-05,2.566667e-05,2.533333e-05,2.5e-05,2.5e-05,2.5e-05,2.5e-05,2.5e-05,2.4e-05,2.4e-05,2.366667e-05,2.333333e-05,2.333333e-05,2.166667e-05,2.1e-05,2.1e-05,2.1e-05,2.1e-05,2.033333e-05,2e-05,1.966667e-05,1.966667e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.733333e-05,1.733333e-05,1.666667e-05,1.666667e-05,1.633333e-05,1.633333e-05,1.6e-05,1.6e-05,1.6e-05,1.566667e-05,1.566667e-05,1.566667e-05,1.566667e-05,1.566667e-05,1.533333e-05,1.466667e-05,1.466667e-05,1.466667e-05,1.466667e-05,1.466667e-05,1.4e-05,1.3e-05,1.3e-05,1.3e-05,1.3e-05,1.3e-05,1.3e-05,1.3e-05,1.3e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.233333e-05,1.2e-05,1.133333e-05,1.133333e-05,1.133333e-05,1.133333e-05,1.1e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05])
arr2=np.array([0.00263,0.001522,0.002293333,0.002127857,0.00177,0.001630476,0.001752917,0.001701538,0.001588077,0.001218889,0.001012857,0.0008446429,0.0007289655,0.0006366667,0.0005946667,0.000502,0.0004183333,0.000389,0.000378,0.0003736667,0.000344,0.00033,0.0003193333,0.0003073333,0.000291,0.0002713333,0.000257,0.0002026667,0.000162,0.0001593333,0.0001453333,0.0001386667,0.0001386667,0.0001386667,0.00012,0.0001093333,0.000109,0.0001063333,0.0001036667,9.266667e-05,8.633333e-05,8.633333e-05,8.566667e-05,8.1e-05,5.766667e-05,4.733333e-05,3.833333e-05,3.833333e-05,3.833333e-05,3.833333e-05,3.533333e-05,3.466667e-05,3.466667e-05,3.466667e-05,3.466667e-05,3.466667e-05,3.266667e-05,3.133333e-05,3.133333e-05,3.133333e-05,3.133333e-05,3.133333e-05,3.133333e-05,3.133333e-05,2.933333e-05,2.933333e-05,2.933333e-05,2.933333e-05,2.933333e-05,2.766667e-05,2.766667e-05,2.7e-05,2.633333e-05,2.633333e-05,2.566667e-05,2.533333e-05,2.5e-05,2.5e-05,2.5e-05,2.5e-05,2.5e-05,2.4e-05,2.4e-05,2.366667e-05,2.333333e-05,2.333333e-05,2.166667e-05,2.1e-05,2.1e-05,2.1e-05,2.1e-05,2.033333e-05,2e-05,1.966667e-05,1.966667e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.833333e-05,1.733333e-05,1.733333e-05,1.666667e-05,1.666667e-05,1.633333e-05,1.633333e-05,1.6e-05,1.6e-05,1.6e-05,1.566667e-05,1.566667e-05,1.566667e-05,1.566667e-05,1.566667e-05,1.533333e-05,1.466667e-05,1.466667e-05,1.466667e-05,1.466667e-05,1.466667e-05,1.4e-05,1.3e-05,1.3e-05,1.3e-05,1.3e-05,1.3e-05,1.3e-05,1.3e-05,1.3e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.266667e-05,1.233333e-05,1.2e-05,1.133333e-05,1.133333e-05,1.133333e-05,1.133333e-05,1.1e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05,1.066667e-05])
# (each leading with -9999,-9999,-9999,-9999,-9999,-9999,-9999,-9999,)

# test-3-7.input all equal :(  niter=3
arr1=np.array([0.0013,0.00164,0.00164,0.00164,0.001596667,0.00137,0.00068,0.00068,0.00068,0.00068,0.00068,0.00068,0.00068,0.0006433333,0.0006433333,0.0006433333,0.0004233333,0.0004233333,0.0004233333,0.0004233333,0.0004233333,0.0002966667,0.0002966667,0.0002966667,0.0002333333,0.0002333333,0.0001666667,0.0001666667,0.0001666667,0.0001666667,0.0001666667,0.0001166667,0.0001166667,0.0001166667,0.0001166667,4.666667e-05,3.333333e-05,3.333333e-05,3.333333e-05,3.333333e-05,3.333333e-05,3.333333e-05,3.333333e-05,3.333333e-05,3.333333e-05,3.333333e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,1.666667e-05,1.666667e-05,1.666667e-05,1.666667e-05,1.666667e-05,1.666667e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05])
arr2=np.array([0.0013,0.00164,0.00164,0.00164,0.001596667,0.00137,0.00068,0.00068,0.00068,0.00068,0.00068,0.00068,0.00068,0.0006433333,0.0006433333,0.0006433333,0.0004233333,0.0004233333,0.0004233333,0.0004233333,0.0004233333,0.0002966667,0.0002966667,0.0002966667,0.0002333333,0.0002333333,0.0001666667,0.0001666667,0.0001666667,0.0001666667,0.0001666667,0.0001166667,0.0001166667,0.0001166667,0.0001166667,4.666667e-05,3.333333e-05,3.333333e-05,3.333333e-05,3.333333e-05,3.333333e-05,3.333333e-05,3.333333e-05,3.333333e-05,3.333333e-05,3.333333e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,3e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,2e-05,1.666667e-05,1.666667e-05,1.666667e-05,1.666667e-05,1.666667e-05,1.666667e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05,1.333333e-05])

In [6]:
navg=1
x1_values=np.arange(11,11+arr1.size-navg)
x2_values=np.arange(11,11+arr2.size-navg)
#take a moving average
y1_values=1./(arr1[navg:]+arr1[:-navg])
y2_values=1./(arr2[navg:]+arr2[:-navg])


I turned off forces, but the effect on collision times was zero to machine precision...

^this means either (i) I'm not in a parameter regime where collision times are significantly effected by forces or (ii) there's a bug in my code

In [7]:
fontsize=18
plt.plot(x1_values, y1_values,lw=2,label='unbiased random walk')
plt.plot(x2_values, y2_values,lw=2,label='attractive forces')
plt.xscale('log')
plt.yscale('log')
# plt.xlim([1e-0,1e3])
# plt.ylim([1e4,6e6])
# plt.title('attractive forces increased exponent\n',fontsize=fontsize)


# plt.title(u'comparison to simulation\nwith two hybrid modes',fontsize=fontsize)
plt.xlabel(r'N',fontsize=fontsize)
plt.ylabel(r'$W_{-2}$ (Hz)', fontsize=fontsize)
# plt.xlabel(r'q (cm$^{-2}$)',fontsize=fontsize)
# plt.ylabel(r'w (Hz cm$^{-2}$)', fontsize=fontsize)
plt.tick_params(axis='both', which='major', labelsize=fontsize)
plt.tick_params(axis='both', which='minor', labelsize=0)
plt.legend(fontsize=fontsize-5,ncol=1,loc='upper left')
plt.show()

In [7]:
# from numpy import linspace
# from scipy.integrate import odeint

# #you can choose a better guess using fsolve instead of 0
# def integral(y, _, F_l, M, cache=[0]):
#     v, preva = y[1], cache[0]
#     #use value for 'a' from the previous timestep
#     F_r = (((1 - preva) / 3) ** 2 + (2 * (1 + preva) / 3) ** 2) * v 
#     #calculate the new value
#     a = (F_l - F_r) / M
#     cache[0] = a
#     return [v, a]

# y0 = [0, 5]
# time = linspace(0., 10., 1000)
# F_lon = 100.
# mass = 1000.

# dydt = odeint(integral, y0, time, args=(F_lon, mass))

In [9]:
# plt.scatter(x=time,y=time*(10-time),c=time,cmap='jet')
# # plt.scatter(x=time,y=dydt[:,0],c=time,cmap='jet')
# # plt.scatter(x=time,y=dydt[:,1])

# # plt.xscale('log')
# # plt.yscale('log')
# # plt.xlim([1e-1,1e1])
# # plt.ylim([1e4,6e4])
# # plt.title('accelerated attraction/annihilation trials only')
# plt.show()

In [10]:
1

1

In [24]:
#load the example data
# os.chdir(nb_dir)
# data_dir='data/out.csv'
# assert ( os.path.exists(data_dir) ) 
# df=pd.read_csv(data_dir,lineterminator=None)

#load a log file
input_folder=f"{nb_dir}/data"
input_fn='example.log'
os.chdir(input_folder)
df=parse_output_log(input_fn, include_inputs=False,printing=False)
Ntrials=df.index.values.shape[0]
Nmin=11
df.drop(columns=[f'{i}' for i in range(Nmin)], inplace=True)
#any duds still there?
assert ( not (df.values<-9000).any() )
df.head()

/home/timothytyree/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in less
  app.launch_new_instance()


11       12       13       14       15       16       17       18  \
0  2.47000  2.47000  2.47000  2.47000  2.47000  2.47000  0.69080  0.56114   
1  1.18204  1.18204  1.18204  1.18204  1.18204  1.18204  1.18204  1.18204   
2  0.81593  0.81593  0.81593  0.81593  0.81593  0.81593  0.81593  0.81593   
3  0.14193  0.14193  0.14193  0.14193  0.14193  0.14193  0.14193  0.14193   
4  2.43258  2.43258  2.43258  2.43258  2.43258  2.43258  2.43258  2.43258   

        19       20  ...       60       61       62       63       64  \
0  0.56114  0.56114  ...  0.06454  0.06454  0.06454  0.06454  0.06454   
1  1.18204  1.18204  ...  0.10980  0.10980  0.02032  0.02032  0.02032   
2  0.81593  0.81593  ...  0.03425  0.03425  0.03425  0.03425  0.03425   
3  0.14193  0.14193  ...  0.03570  0.03570  0.03570  0.03570  0.02346   
4  2.43258  2.43258  ...  0.00877  0.00877  0.00877  0.00877  0.00877   

        65       66       67       68       69  
0  0.06454  0.06454  0.06454  0.06454  0.02795  
1  0.02032  0.02032  0.02032  0.02032  0.02032  
2  0.03425  0.03425  0.03425  0.03425  0.03425  
3  0.02346  0.02346  0.02346  0.02346  0.02346  
4  0.00877  0.00877  0.00877  0.00877  0.00877  

[5 rows x 59 columns]

# notes/eqns for ou process

Suppose we have two particles that have been nearest neighbors since time, $t=0$.
Let $X_t=$ the distance between two particles at time $t$.  We may then model $X_t$ reverting to (from) some preferred distance, $x_0\ge0$ by setting $\varkappa>0$ ($\varkappa<0$) in 
$$
dX_t=\varkappa(x_0-X_t)dt+\sqrt{2D}dW_t.
$$

For $\varkappa>0$, the expected distance is 
$$
\mathbb{E}[X_t]=X_0e^{-\varkappa t} + x_0(1-e^{-\varkappa t})
$$
and the covariance is
$$
\text{cov}(x_s,x_t)=\frac{D}{\varkappa}\Big(e^{-\varkappa |t-s|}-e^{-\varkappa (t+s)}\Big).
$$


# DONT: dev add-result.pl

# TODO: dev routine that uses dask to do a sweep over N
HINT: descend from Nmax to Nmin

In [1]:
#######################################################
### TODO: implement in PYTHON (and later) with DASK ###
#######################################################
#TODO: open a file in append mode
#TODO: print N, from Nmax descending to $Nmin
#TODO: write params.input for this Ntips
#TODO: system(./xrun < params.input)
#TODO: change params.input to params/params_N_{N}.input
#TODO: do all ^this in dask, but not with a daskbag... this should update a preallocated vector as tasks are finishedd
#HINT: preallocation means save in current increasing N format

In [3]:
# #DONT(do perl stuff in python): initialize line 
# i=37
# string.split(',')[i]

In [8]:
#TODO(goal): get item_lst from a daskbag for item = (n,Tavg)
Nmax=60;Nmin=6
n_values=np.arange(Nmax,Nmin,-1)
n_values

array([60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44,
       43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27,
       26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10,
        9,  8,  7])

In [27]:
#template arguments for simulation
x=np.array([0.1, 2, 5, 500, 0., 0., 1e-5, 1e-5, 8, 500, 1234, 0, 0, 0, 0])
log_dir="/home/timothytyree/Documents/GitHub/bgmc/c/ou"
os.chdir(log_dir)
#make routine that generates Tavg for a given n
def routine(n):
    fn_out=f"Log/out_n_{n}.output"
    #TODO: integrate return_CollTime with routine
    os.system(f"/return_CollTime.x < 1-control.input | grep 'Tavg=' | grep -Eo '[+-]?[0-9]+([.][0-9]+)?' >  Log/1-control.output")
#     os.system(f"./xrun.sh ${x[0]} ${x[1]} ${x[2]} ${x[3]} ${x[4]} ${x[5]} ${x[6]} ${x[7]} {n} ${x[9]} ${x[10]} ${x[11]} > {fn_out}")
    #TODO: parse fn_out for Tavg
    retval=os.system(f'grep "Tavg=" {fn_out}')#' #| grep -Eo "[+-]?[0-9]+([.][0-9]+)?"')
#     retval=os.system(f'grep "Tavg=" {fn_out} #| grep -Eo "[+-]?[0-9]+([.][0-9]+)?"')
    #TODO: return Tavg
    return retval

In [28]:
n=32
# os.system(f"./xrun.sh ${x[0]} ${x[1]} ${x[2]} ${x[3]} ${x[4]} ${x[5]} ${x[6]} ${x[7]} {n} ${x[9]} ${x[10]} ${x[11]} > a.out")
routine(n)

0

In [26]:
!ls

1-test.input	 output.txt		       runs
1-test.output	 params.input		       src
add-results.pl	 post_process.sh	       summarize.sh
a.txt		 prepare-CollTimes.pl	       test-3.input
clean-log.sh	 python			       test-4.input
CommonDefines.h  requirements.sh	       test-5.input
dev		 return_CollTime.c	       test-6.input
fktest.input	 return_CollTimes_Archived.c   test.input
gcc.sh		 return_CollTimes.c	       testpl.sh
key.input	 return_CollTimes.sh	       test.sh
lib		 return-CollTimes.submit       test.txt
local-main.sh	 return-CollTimes-test.submit  testy.sh
Log		 return_CollTimes.x	       xrun.sh
myoutput.txt	 return_CollTime.x


In [34]:
import numpy as np
niter=1500 #trials per worker
r_values=np.array([0.1,1.0])#,0.2,0.3,0.4,0.5,.6,.7,.8,.9,1.,2.])#cm
D_values=np.array([2.,20.])#0.2,1.0,1.5,2.0,3.,4.,5.])#cm^2/s
A_values=np.array([25.])#20.25,25,39,50,56.25,100,156.25,189,250])[::-1]#cm^2
L_values=np.sqrt(A_values)#cm
kappa_values=np.array([500,1500])#5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100,250,500])#1/s
varkappa_values=np.array([-5,-1.5,-0.2,0,0.2,1.5,5])#1/s
x0_values=np.array([0.,0.1,0.2,0.3,0.4,0.5,1.0])#cm
Dt_values=np.array([1e-5,1e-2])#10**-i for i in range(6)])
dt=1e-5
Nmax=100
num_trials_per_setting=1
reflect_values=np.array([0])
set_second_values=np.array([0])
no_repulsion_values=np.array([0,1])
no_attraction_values=np.array([0,1])
#iterate over settings, scheduling the longest jobs first
count=0
for Dt in Dt_values:
	for set_second in set_second_values:
		for reflect in reflect_values:
			for r in r_values:
				for D in D_values:
					for L in L_values:
						for kappa in kappa_values:
							for varkappa in varkappa_values:
								for x0 in x0_values:
									for no_repulsion in no_repulsion_values:
										for no_attraction in no_attraction_values:
											num_trials=0
											while num_trials<num_trials_per_setting:
												num_trials+=1
												count=count+1
# 												print(f"{r} {D} {L} {kappa} {varkappa} {x0} {Dt} {dt} {Nmax} {niter} {reflect} {set_second} {no_repulsion} {no_attraction}")
print(count)


3136
